# 1) Configuration de PRAW (Python Reddit Api Wrapper)
## !! A faire avant de pouvoir utiliser n'importe laquelle des parties suivantes !!

In [1]:
import praw               # API pour reddit
import pandas as pd       # pour afficher les dictionnaires sous forme de tableaux
import datetime as dt     # Pour convertir la date au bon format
import os
import sys

In [2]:
# Créer la variable de configuration pour accéder aux requêtes vers reddit
reddit = praw.Reddit(client_id='BEjar6X3GYV5Vw', \
                     client_secret='lg6D6DXG14FH4kHtGVt7cins5OY', \
                     user_agent='feelcrypt', \
                     username='feelcrypt', \
                     password='spiderminute38')

# 2) Générer la liste des posts à traiter
## !! Ne lancer qu'une fois au totale !!

Cette partie va récupérer les 500 top posts du subreddit défini et sauvegarder uniquement les id de ces posts dans une liste de post à traiter (un fichier txt)

In [3]:
# Variables de configuration globale
nb_top_posts = 5  # nombre de posts selectionnés parmi les premiers (limite = 500)
subreddit_title = 'bitcoin' # Définir le titre du subreddit que l'on va cibler

In [4]:
# Créer un dossier pour le subreddit

# Générer le chemin en fonction du subreddit traité
script_dir = %pwd
subreddit_dir = script_dir + '\\'+ subreddit_title +'\\' #
try:
    os.makedirs(subreddit_dir)
    print('Dossier ' + subreddit_dir + ' créé')
except FileExistsError:
    print('Dossier déjà existant')
    pass

Dossier C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V2\bitcoin\ créé


In [5]:
# Vérifier si le dossier comments (chemin relatif) existe, sinon le créer
comments_dir = subreddit_dir + 'comments'
try:
    os.makedirs(comments_dir)
    print('Dossier ' + comments_dir + ' créé')
except FileExistsError:
    print('Dossier déjà existant')
    pass

Dossier C:\Users\Louis\feelcrypt\FeelCrypt\scrapping_reddit\scrapping_V2\bitcoin\comments créé


In [6]:
# Créer la variable sur le subreddit btc
subreddit = reddit.subreddit(subreddit_title)

In [7]:
# Récupérer les n premiers posts de notre subreddit de notre liste
top_posts = []

for post in subreddit.top(limit = nb_top_posts):
    top_posts.append(post)

In [8]:
# Enregistrer les id dans une liste
top_posts_id = []
for post in top_posts:
    top_posts_id.append(post.id)

In [9]:
# Enregistrer la liste des id dans un fichier txt
id_todo_path = subreddit_dir +'id_todo.txt'
top_posts_id_file = open(id_todo_path,'w')
for post_id in top_posts_id:
     top_posts_id_file.write(post_id)
     top_posts_id_file.write('\n')
top_posts_id_file.close()

In [10]:
# Fichier pour compter le nombre de commentaires total
# Le créer et le réinitialiser à 0 s'il existe déjà
nb_file = open('comments_counter.txt','w').close()
nb_file = open('comments_counter.txt','w')
nb_file.write(str(0))
nb_file.close()

## 6.2) Traiter les top posts

In [11]:
# Variables

scrap_all_comments = False
# Si false : prend seulement les 32 premiers commentaires de chaque post
# Si true : prend tous les commentaires du post

scrap_responses_to_comments = True
# Si false : ne prend que les réponses directes au post
# Si true : prend également en compte les réponses aux commentaires

In [12]:
# Fonction de conversion pour la date avec timestamp
def get_date(created):
    return dt.date.fromtimestamp(created)

In [13]:
# Ouvrir la liste de posts pas encore faits
with open(id_todo_path) as file:
  id_todo = file.readlines()

### Ne pas oublier de remettre le replace_more limit = NONE

In [15]:
# Traiter chaque id du fichier todo

while id_todo:  # Vérifier si la liste n'est pas vide
    
    # Récupérer le post
    post = reddit.submission(id = id_todo[0])
    current_post_id = post.id
    print('-----------------------------------')
    print('Starting working on post : ' + str(current_post_id))
    
    ## Récupérer les commentaires du poste
    comments_dict = { "created_utc":[], "body":[]} #Réinitialiser le dictionnaire
    
    #se débarasser récursivement de la limite de 32 commentaires par requête
    if scrap_all_comments:
        post.comments.replace_more(limit=None)  # prendre en compte les commentaires supp
    else:
        post.comments.replace_more(limit=0)    # ignorer les commentaires supp
    
    # Récupérer la liste des commentaires
    if scrap_responses_to_comments:
        comments_list = post.comments.list()
    else:
        comments_list = post.comments
        
    # Remplir le dictionnaire de commentaires avec la liste  
    for comment in comments_list:
        comments_dict["created_utc"].append(comment.created_utc)
        comments_dict["body"].append(comment.body)
        
    # mettre la data au format pandas (qui permet de faire un "tableur" à partir du dictionnaire)
    comments_data = pd.DataFrame(comments_dict)
    
    # Créer la liste des dates converties et la sauvgarder dans la variable _timstamp
    # created_utc est la colonne contenant les dates au mauvais format
    _timestamp = comments_data["created_utc"].apply(get_date)

    # ajouter la liste à une nouvelle colonne appelée timestamp
    comments_data = comments_data.assign(date = _timestamp)
    
    # Supprimer la colonne du temps inutile
    comments_data = comments_data.drop(columns="created_utc")

    # Déplacer la date en première position
    colonnes = comments_data.columns.tolist()
    colonnes = colonnes[-1:] + colonnes[:-1]
    comments_data = comments_data[colonnes]
    
    # Trier par la colonne date
    comments_sorted = comments_data.sort_values(by=['date'])
    
    ## Extraire plusieurs dataframe qui représentent chacun une date avec tous les commentaires dedans
    # Récupérer la liste des dates et indexer par dates
    comments_sorted.set_index(keys=['date'], drop=False,inplace=True)
    dates = comments_sorted['date'].unique().tolist()
    
    # Enregistrer dans une liste contenant chaque dataframe (1 data frame = 1 date)
    comments_splitperday = []
    for date in dates:
        comments_per_day = pd.DataFrame(comments_sorted.loc[comments_sorted.date == date])
        comments_splitperday.append(comments_per_day)
    
    # Enregistrer chaque dataframe dans un fichier csv
    for dataframe in comments_splitperday:
        # Récupérer la date du dataframe supprimer la colonne date
        date = str(dataframe.date.iloc[0])
        dataframe = dataframe.drop(columns="date")
        csv_path = comments_dir + "\\" + date + '.csv'
        
        # Enregistrer au format csv avec pour nom la date
        dataframe.to_csv(csv_path, ';', mode='a', index=False, header=False) 
        
    # Retirer le premier id de la liste, le laisser à la fin, si le while est interrompu, il sera retiré alors que le post n'aura pas été traité
    id_todo.pop(0)
    
    # Ajouter l'id du post lu dans la liste des id_done
    id_done_path = subreddit_dir + "id_done.txt"
    with open(id_done_path, "a") as file:
        file.write(current_post_id)
       
    # Enregistrer la nouvelle liste des id_todo dans le fichier txt (ou le créer s'il n'existe pas encore)
    with open(id_todo_path, "r") as file:
        data = file.read()
    with open(id_todo_path, "w") as file:
        for post_id in id_todo:
            file.write(post_id)
    
    # Ajouter le nb de commentaires traités dans le compteur
    nb_file = open('comments_counter.txt','r')
    total_comments = int(nb_file.read())
    total_comments += len(comments_data.index)
    nb_file.close()
    nb_file = open('comments_counter.txt','w').close()
    nb_file = open('comments_counter.txt','w')
    nb_file.write(str(total_comments))
    nb_file.close()
    
    print(str(len(comments_data.index)) + " comments from post " + str(current_post_id))
    print('total comments saved = ' + str(total_comments))
else:
    print('---------------------------------------------------------')
    print("La liste est vide, tout a été traité")

-----------------------------------
Starting working on post : 7lo56s

475 comments from post 7lo56s

total comments saved = 7976
-----------------------------------
Starting working on post : 7mgy26

467 comments from post 7mgy26

total comments saved = 8443
-----------------------------------
Starting working on post : 7aq0fx

486 comments from post 7aq0fx

total comments saved = 8929
-----------------------------------
Starting working on post : 6tcg99

483 comments from post 6tcg99

total comments saved = 9412
-----------------------------------
Starting working on post : 7gzr64

459 comments from post 7gzr64

total comments saved = 9871
-----------------------------------
Starting working on post : 953ays

459 comments from post 953ays

total comments saved = 10330
-----------------------------------
Starting working on post : 7gonjk

468 comments from post 7gonjk

total comments saved = 10798
-----------------------------------
Starting working on post : 7jri98

471 comments from

KeyboardInterrupt: 

In [13]:
len(subreddit.comments)

TypeError: object of type 'CommentHelper' has no len()